In [82]:
from importlib import reload
import numpy as np
import scipy.linalg
import linalg
reload(linalg)
import pywren
reload(pywren)
import math
import itertools
import time
import cloudpickle
import importlib
import sys
from sklearn.datasets import fetch_mldata
from itertools import groupby
from operator import itemgetter
from sklearn.kernel_approximation import RBFSampler, Nystroem
from sklearn import metrics
import sys
import boto3
import concurrent.futures as fs
reload(boto3)
import os
from numba import jit
from scipy.linalg import LinAlgError

In [2]:
@jit(nopython=True)
def __calculate_res(R, num_blocks, b, K_block, x, block_size, n):
    for b2 in range(int(num_blocks)):
        if b2 == b: continue
        s =  b2*block_size
        e = min((b2+1)*block_size, n)
        if (np.all(x[s:e, :] == 0)): continue
        Kbb2 = K_block[s:e]
        a = np.dot(Kbb2.T, x[s:e, :])
        R += a
    return R

In [10]:
%time x = np.random.randn(*y_train_one_hot.shape)
%time X_block  = np.random.randn(X_train.shape[0], 4096)

CPU times: user 1min 38s, sys: 2.26 s, total: 1min 40s
Wall time: 1min 40s
CPU times: user 4min 15s, sys: 9.51 s, total: 4min 24s
Wall time: 4min 24s


In [94]:
x =  X_block[:, :1000].astype('float32')

In [91]:
X_block = X_block.astype('float32')

In [93]:
R = np.zeros((4096, 1000)).astype('float32')

In [95]:
%time __calculate_res(R, 313, 11, X_block, x, 4096, X_train.shape[0])

CPU times: user 8min 8s, sys: 1min 45s, total: 9min 54s
Wall time: 9.69 s


array([[  1.27659825e+06,  -4.16446686e+00,   5.32080322e+02, ...,
         -1.78844678e+03,   1.25570703e+03,  -1.89831555e+03],
       [ -4.16446686e+00,   1.27674412e+06,  -1.64152893e+03, ...,
          1.22419958e+03,  -1.28372217e+03,   1.89988049e+03],
       [  5.32080322e+02,  -1.64152893e+03,   1.27610588e+06, ...,
          2.31576401e+02,   1.78166748e+03,  -1.04506836e+03],
       ..., 
       [  1.09124829e+03,  -3.16828308e+02,  -2.82688599e+02, ...,
          1.04686499e+03,  -1.74970551e+02,  -7.71156982e+02],
       [  6.14441910e+01,   9.90693665e+02,   5.97356079e+02, ...,
          9.59382263e+02,  -1.08257275e+03,   1.93228973e+02],
       [ -1.91252222e+03,  -1.79226880e+03,  -4.59195129e+02, ...,
         -2.65430664e+03,  -2.46139404e+03,   1.28208740e+03]], dtype=float32)

In [61]:
def block_kernel_solve(K, y, epochs=1, max_iter=313, block_size=4096, num_blocks=313, lambdav=0.1, verbose=True, prc=lambda x: x, workers=22):
        '''Solve (K + \lambdaI)x = y
            in a block-wise fashion
        '''
        labels = np.argmax(y, axis=1)
        with fs.ProcessPoolExecutor(max_workers=workers) as executor:
            mmap_locs = ["/dev/shm/block0", "/dev/shm/block1"]
            mmap_loc = mmap_locs[0]
            # compute some constants
            x = np.zeros(y.shape, 'float32')
            i = 0
       

            K_block_future = fast_kernel_column_block_async(K_sharded, 0, mmap_loc=mmap_loc, executor=executor, workers=workers)
            
            for e in range(epochs):
                    y_hat = np.zeros(y.shape)
                    for b in range(int(num_blocks)):
                            iter_start = time.time()
                            if (i > max_iter):
                                return x

                            mmap_loc = mmap_locs[i%2 == 0]
                            print("Grabbing this block from oven")
                            s = time.time()
                            K_block = load_mmap(*K_block_future.result())
                            e = time.time()
                            print("Block spent {0} seconds in oven".format(e - s))
                            print("Putting next Block in oven")
                            s = time.time()
                            K_block_future = fast_kernel_column_block_async(K_sharded, (b+1)%num_blocks, mmap_loc=mmap_loc, executor=executor, workers=workers)
                            # pick a subset of the kernel matrix (note K can be mmap-ed)
                            e = time.time()
                            print("Took {0} seconds to put block in oven".format(e - s))
                            b_start =  b*block_size
                            b_end = min((b+1)*block_size, K.shape[1])
                            y_block = y[b_start:b_end, :]
                            start = time.time()
                            R = np.zeros((b_end - b_start, y.shape[1]), 'float32')
                            __calculate_res(R, num_blocks, b, K_block, x, block_size, K.shape[0])
                            end = time.time() 
                            print("Residual time {0}".format(end - start))
                            start = time.time()
                            Kbb = K_block[b_start:b_end, :].astype('float64')
                            print(Kbb.shape)
                            # Add term to regularizer
                            idxes = np.diag_indices(Kbb.shape[0])
                            try: 
                                Kbb[idxes] += lambdav
                                x_block = scipy.linalg.solve(Kbb, y_block - R, sym_pos=True)
                                Kbb[idxes] -= lambdav
                                # update model
                                x_block = x_block.astype('float32')
                                x[b_start:b_end] = x_block
                                t = time.time()
                                print("Residual is {0}".format(np.linalg.norm(y_block - R)))
                                '''
                                y_hat += K_block.dot(x_block)
                                acc = metrics.accuracy_score(np.argmax(y_hat, axis=1), labels)
                                print("Iteration {0}, Training Accuracy {1}".format(i, acc))
                                e = time.time()
                                print("Calculating accuracy took {0} secs ".format(e -s))
                                '''
                            except LinAlgError as e:
                                print("Singular matrix in block {0} with reg {1}".format(b, lambdav))
                            iter_end = time.time()
                            print("Iteration {0} took {1} seconds".format(i, iter_end - iter_start))
                            i += 1
            return x
        
def load_mmap(mmap_loc, mmap_shape, mmap_dtype):
    return np.memmap(mmap_loc, dtype=mmap_dtype, mode='r+', shape=mmap_shape)

def fast_kernel_column_block_async(K, col_block, executor=None, workers=22, mmap_loc="/dev/shm/block0", wait=False):
    if (executor == None):
        executor = fs.ProcessPoolExecutor(max_workers=workers)
    num_blocks = 313
    if ((col_block+1)*K.shard_size_1 > K.shape[1]):
        block_width = (K.shape[1] - col_block*K.shard_size_1)
    else:
        block_width = (K.shard_size_1)
        
    mmap_shape = (K.shape[0],  block_width)
    s = time.time()
    if (not os.path.isfile(mmap_loc)):
        np.memmap(mmap_loc, dtype='float32', mode='w+', shape=mmap_shape)
    e = time.time()
    futures = []
    chunk_size = int(np.ceil(num_blocks/workers))
    chunks = linalg.chunk(list(range(num_blocks)), chunk_size)
    for c in chunks:
        futures.append(executor.submit(K.get_blocks_mmap, c, col_block, mmap_loc, mmap_shape))
    return futures[0]

        
def fast_kernel_column_block_get(K, col_block, workers=21):
    num_blocks = 313
    with fs.ProcessPoolExecutor(max_workers=workers) as executor:
        futures = []
        chunk_size = int(np.ceil(num_blocks/workers))
        chunks = list(linalg.chunk(list(range(num_blocks)), chunk_size))
        print(len(chunks))
        for c in chunks:
            futures.append(executor.submit(K.get_blocks, c, col_block))
        fs.wait(futures)
        return np.vstack(list(map(lambda x: x.result(), futures)))

      

In [5]:
notebook_stdout = sys.stdout
notebook_stderr = sys.stderr
terminal_stdout = open('/dev/stdout', 'w')
terminal_stderr = open('/dev/stderr', 'w')

In [6]:
train_data = np.load("imagenet_features_alexnet_fc7.npz")

In [7]:
X_train = train_data["X_train"]
y_train = train_data["y_train"]
y_train_one_hot = np.eye(1000)[y_train]

In [68]:
X_test = train_data['X_test']

In [78]:
y_test = train_data["y_test"]

In [8]:
%time X_train_sharded = linalg.ShardedMatrix("imagenet-train", data=X_train, shard_size_0=4096, bucket="imagenet-raw", reshard=False, replication_factor=5, n_jobs=8)

CPU times: user 92 ms, sys: 12 ms, total: 104 ms
Wall time: 284 ms


###### X_train_sharded

In [9]:
# set up pywren
gamma = 1e-3
pwex = pywren.default_executor()
num_blocks = 313
tasks_per_job = 5
num_jobs = 2500

In [66]:
chunked_blocks = linalg.generate_chunked_block_pairs(num_blocks, tasks_per_job, num_jobs)

In [64]:
for c in chunked_blocks:
    %time kernel_futures = pwex.map(lambda x: linalg.compute_rbf_kernel_blockwise(x, X_train_sharded, gamma=1e-3), c)
    %time pywren.wait(kernel_futures)    
K_sharded = linalg.compute_linear_kernel_blockwise([(0,0)], X_train_sharded)[1]

CPU times: user 1min 14s, sys: 4.9 s, total: 1min 19s
Wall time: 29.7 s
{'runtime_s3_bucket_used': 'ericmjonas-public', 'exception': "('OUTATIME', 'Process executed for too long and was killed')", 'data_download_time': 0.14664888381958008, 'runtime_s3_key_used': '87950c-pywren.runtime/pywren_runtime-3.6-default.tar.gz.0029', 'setup_time': 0.1941220760345459, 'start_time': 1490575645.93137, 'callset_id': 'a29cd728-ff4f-4d41-929c-196578119fc1', 'func_key': ['imagenet-raw', 'pywren.jobs/a29cd728-ff4f-4d41-929c-196578119fc1/func.json'], 'call_id': '00480', 'status_key': ['imagenet-raw', 'pywren.jobs/a29cd728-ff4f-4d41-929c-196578119fc1/00480/status.json'], 'func_download_time': 0.13198590278625488, 'exception_traceback': 'Traceback (most recent call last):\n  File "/var/task/wrenhandler.py", line 296, in generic_handler\n    "Process executed for too long and was killed")\nException: (\'OUTATIME\', \'Process executed for too long and was killed\')\n', 'output_key': ['imagenet-raw', 'pywren

In [83]:
K_sharded = linalg.compute_rbf_kernel_blockwise([(0,0)], X_train_sharded, gamma=1e-3)[1]

In [56]:
K_sharded = kernel_futures[0]

NameError: name 'kernel_futures' is not defined

In [39]:
K_sharded.get_block(1,0)

array([[  666.1619873 ,   315.49255371,   717.11315918, ...,
          505.31872559,   607.21002197,   304.80075073],
       [  526.69262695,   333.09085083,   830.56774902, ...,
         1389.02783203,   844.98492432,   790.25201416],
       [ 1212.43017578,   595.01367188,   797.11987305, ...,
         1002.1751709 ,   664.14239502,   880.27789307],
       ..., 
       [  531.60418701,   936.97436523,   732.84973145, ...,
          441.60214233,  1065.49291992,   654.55810547],
       [  376.57577515,   147.31192017,   581.53985596, ...,
          902.98059082,   563.4979248 ,   885.77587891],
       [  461.48388672,   335.40252686,   686.89154053, ...,
          821.79394531,   847.71313477,   570.93084717]], dtype=float32)

In [85]:
X_train[4096:4096*2, :].dot(X_train[:4096, :].T)

array([[  666.1619873 ,   315.49255371,   717.11315918, ...,
          505.31872559,   607.21002197,   304.80075073],
       [  526.69262695,   333.09085083,   830.56774902, ...,
         1389.02783203,   844.98492432,   790.25201416],
       [ 1212.43017578,   595.01367188,   797.11987305, ...,
         1002.1751709 ,   664.14239502,   880.27789307],
       ..., 
       [  531.60418701,   936.97436523,   732.84973145, ...,
          441.60214233,  1065.49291992,   654.55810547],
       [  376.57577515,   147.31192017,   581.53985596, ...,
          902.98059082,   563.4979248 ,   885.77587891],
       [  461.48388672,   335.40252686,   686.89154053, ...,
          821.79394531,   847.71313477,   570.93084717]], dtype=float32)

In [ ]:
gamma = 1e-3
lambdav = 1e-4

In [13]:
sys.stdout = terminal_stdout


In [33]:
%time XtX = X_train.T.dot(X_train)

In [30]:
%time Xty = X_train.T.dot(y_train_one_hot)

In [31]:
lambdav = 1e-8

In [34]:
diag = np.diag_indices(XtX.shape[0])
XtX[diag] += lambdav
model = scipy.linalg.solve(XtX, Xty)
XtX[diag] -= lambdav

In [35]:
metrics.accuracy_score(np.argmax(X_train.dot(model), axis=1), y_train)

0.61269295884143127

In [66]:
np.save("X_1_epoch", X_1_epoch)

In [84]:
sys.stdout = terminal_stdout
X_1_epoch = block_kernel_solve(K_sharded, y_train_one_hot, epochs=1, lambdav=1)

In [59]:
prc, U = linalg.make_smart_precondition(X_train, gamma=gamma, lambdav=lambdav, n_components=4096)

Computing Nystroem svd
Computing Nystroem features
Computing ZTZ
Computing Cholesky
(4096, 1281167)


In [ ]:

args = X_3
args = list(args)

In [115]:
args[0] = np.zeros((4096, y_train_one_hot.shape[1]))
args

In [119]:
__calculate_res(*args)

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [33]:
x = np.random.randn(4096,4096)

In [38]:
np.sum([x,x], axis=0)

array([[ 1.00043242,  1.88256286,  2.23652953, ...,  0.71320838,
        -2.97133385,  0.24525873],
       [-1.14752926,  0.28331297, -0.35583467, ...,  1.58477624,
        -0.96406481,  0.1548442 ],
       [ 0.13228222, -1.50739475,  0.50730763, ..., -4.60623277,
        -3.08502852, -2.83105076],
       ..., 
       [ 1.97830904, -1.07704047, -0.94826343, ..., -1.26330201,
         0.51951157,  0.33207813],
       [ 2.35859914,  2.24013857,  1.37480261, ..., -2.56782584,
         2.01003214, -0.48774642],
       [ 3.76238426,  0.18478296,  1.25920478, ..., -2.32657978,
         0.11346072,  3.16147817]])

In [39]:
x + x

array([[ 1.00043242,  1.88256286,  2.23652953, ...,  0.71320838,
        -2.97133385,  0.24525873],
       [-1.14752926,  0.28331297, -0.35583467, ...,  1.58477624,
        -0.96406481,  0.1548442 ],
       [ 0.13228222, -1.50739475,  0.50730763, ..., -4.60623277,
        -3.08502852, -2.83105076],
       ..., 
       [ 1.97830904, -1.07704047, -0.94826343, ..., -1.26330201,
         0.51951157,  0.33207813],
       [ 2.35859914,  2.24013857,  1.37480261, ..., -2.56782584,
         2.01003214, -0.48774642],
       [ 3.76238426,  0.18478296,  1.25920478, ..., -2.32657978,
         0.11346072,  3.16147817]])

In [50]:
scipy.linalg.LinAlgError

numpy.linalg.linalg.LinAlgError

In [ ]:
KTest = linalg.computeRBFGramMatrix(X_test,X_train,gamma=1e-3)

In [ ]:
test_preds = KTest.dot(X_1_epoch)

In [87]:
test_labels_pred = np.argmax(test_preds, axis=1)

In [88]:
test_labels_pred

array([303, 987, 491, ...,  35, 351, 573])

In [89]:
metrics.accuracy_score(y_test, test_labels_pred)

0.48330000000000001

In [1]:
def softmax_gn_old(X_train, y_train, X_test, y_test, multiplier=1e-2, numiter=50, verbose=False):
        ''' Implementation of gauss-newton quassi-newton optimization algorithm
            with softmax objective
        '''
        X_train = X_train.reshape(X_train.shape[0], -1)
        X_test = X_test.reshape(X_test.shape[0], -1)
        gmat = (1.0/X_train.shape[0])*(X_train.T.dot(X_train))
        lambdav = multiplier*np.trace(gmat)/gmat.shape[0]
        gmat = gmat + lambdav*np.eye(gmat.shape[0]);
        w = np.zeros((10, gmat.shape[0]))
        num_samples = X_train.shape[0]
        onehot = lambda x: np.eye(10)[x]
        y_train_hot = np.array(map(onehot, y_train))
        y_test_hot = np.array(map(onehot, y_test))
        for k in range(numiter):
            train_preds  = w.dot(X_train.T).T # 60000 x 10
            train_preds = train_preds - np.max(train_preds, axis=1)[:,np.newaxis]
            train_preds = np.exp(train_preds)
            train_preds = train_preds/(np.sum(train_preds, axis=1)[:,np.newaxis])
            train_preds = y_train_hot - train_preds
            grad = (1.0/num_samples)*(X_train.T.dot(train_preds).T) - lambdav*w
            w = w + (np.linalg.solve(gmat, grad.T)).T
            y_train_pred = np.argmax(w.dot(X_train.T).T, axis=1)
            y_test_pred = np.argmax(w.dot(X_test.T).T, axis=1)
            train_acc = metrics.accuracy_score(y_train, y_train_pred)
            test_acc = metrics.accuracy_score(y_test, y_test_pred)
            if (verbose):
              print("Iter: {0}, Train Accuracy: {1}, Test Accuracy: {2}".format(k, train_acc, test_acc))
        return y_train_pred, y_test_pred, w
